# Dimensión sede

In [1]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from datetime import date

### Conexión con la base de datos

In [2]:
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")
#print(url_oltp)
#print(url_olap)
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

### Módulo de extracción

In [3]:
dimension_sede = pd.read_sql_table('sede', oltp_conn)
table_ciudad = pd.read_sql_table('ciudad', oltp_conn)
table_departamento = pd.read_sql_table('departamento', oltp_conn)
#dim_cliente = pd.read_sql_table('cliente', oltp_conn)
#dim_sede.info()
#table_ciudad.head()
#table_departamento.head()

### Módulo de transformación

In [4]:
table_departamento = table_departamento.rename(columns={'nombre' : 'departamento'})
table_ciudad = table_ciudad.rename(columns={'nombre' : 'ciudad'})
table_ciudad = pd.merge(table_ciudad, table_departamento, how='inner') # reemplazo de las llaves foraneas de la tabla ciudad (elimina departamento_id)
table_ciudad = table_ciudad.drop(columns=['departamento_id']) # organizar
#table_ciudad.head()

# reemplazar llaves foraneas de sede usando ciudad_id y cliente_id
dimension_sede = dimension_sede.rename(columns={'nombre' : 'nombre_sede'})
dimension_sede = pd.merge(dimension_sede, table_ciudad, how='inner', left_on='ciudad_id', right_on='ciudad_id')
dimension_sede = dimension_sede.drop(columns=['ciudad_id', 'cliente_id']) # pa qué sirve cliente_id aquí?
print(dimension_sede.shape[0])
dimension_sede.sort_values(by='sede_id')

52


,sede_id,nombre_sede,direccion,telefono,nombre_contacto,ciudad,departamento
14,1,Sede principal - Cliente1,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
15,2,sede aux - cliente 1,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
49,3,TORRES DE MARACAIBO,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
50,4,INGENIO,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
16,5,VASQUEZ COBO,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
24,6,CAPITOLIO,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
17,7,VILLACOLOMBIA,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
19,8,PRINCIPAL / OCCIDENTE,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
18,9,PRINCIPAL/ ESENSA,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA
0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,VALLE DEL CAUCA


### Módulo de carga a la bodega de datos